This notebook is a result of this crazy approach: https://www.kaggle.com/jazivxt/all-this-over-a-dog/ Wanted to try a variant of this before implementing my approach.

In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesRegressor

In [16]:
train_data=pd.read_csv('../input/train.csv')
test_data=pd.read_csv('../input/test.csv')
structure_data=pd.read_csv('../input/structures.csv')
print("Done")

Done


In [17]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structure_data, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train_data = map_atom_info(train_data, 0)
train_data = map_atom_info(train_data, 1)
test_data = map_atom_info(test_data, 0)
test_data = map_atom_info(test_data, 1)
del structure_data

In [18]:
train_data.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [19]:
test_data.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


In [20]:
mulliken_data = pd.read_csv('../input/mulliken_charges.csv').rename(columns={'atom_index':'atom_index_0'})
train_data = pd.merge(train_data, mulliken_data, how='left', on=['molecule_name', 'atom_index_0'])
test_data = pd.merge(test_data, mulliken_data, how='left', on=['molecule_name', 'atom_index_0'])
del mulliken_data

scc_data = pd.read_csv('../input/scalar_coupling_contributions.csv')
train_data = pd.merge(train_data, scc_data, how='left', on=['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])
test_data = pd.merge(test_data, scc_data, how='left', on=['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])
del scc_data

In [21]:
train_data.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,83.0224,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,-11.0347,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,-11.0325,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,-11.0319,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,83.0222,0.254585,1.25861,0.272013


In [22]:
test_data.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge,fc,sd,pso,dso
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,NaN,NaN,NaN
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,NaN,NaN,NaN
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,NaN,NaN,NaN
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,NaN,NaN,NaN


In [24]:
train_data.drop(['id', 'molecule_name', 'atom_0', 'atom_1', 'atom_index_0', 'atom_index_1'], axis=1, inplace=True)
test_data.drop(['id','molecule_name', 'atom_0', 'atom_1', 'atom_index_0', 'atom_index_1'], axis=1, inplace=True)

In [26]:
train_p0 = train_data[['x_0', 'y_0', 'z_0']].values
train_p1 = train_data[['x_1', 'y_1', 'z_1']].values
test_p0 = test_data[['x_0', 'y_0', 'z_0']].values
test_p1 = test_data[['x_1', 'y_1', 'z_1']].values

train_data['dist'] = np.linalg.norm(train_p0 - train_p1, axis=1)
test_data['dist'] = np.linalg.norm(test_p0 - test_p1, axis=1)

train_data['dist_to_type_mean'] = train_data['dist'] / train_data.groupby('type')['dist'].transform('mean')
test_data['dist_to_type_mean'] = test_data['dist'] / test_data.groupby('type')['dist'].transform('mean')

In [28]:
def features(df, df2):
    for c in ['mulliken_charge', 'fc', 'sd', 'pso', 'dso', 'dist', 'dist_to_type_mean']:
        for agg in ['min', 'max', 'sum', 'mean']:
            tmp = eval('df.groupby(["type"], as_index=False)[c].' + agg + '().rename(columns={"' + c + '":"' + agg + '"+ "_" +"' + c + '"})')
            df = pd.merge(df, tmp, how='left', on=['type'])
            df2 = pd.merge(df2, tmp, how='left', on=['type'])
    return df, df2

train_data, test_data = features(train_data, test_data)
print(train_data.shape, test_data.shape)

(4658147, 43) (2505542, 42)


In [29]:
def features(df):
    for c in ['_0', '_1']:
        col = [c1 + c  for c1 in ['x','y','z']]
        for agg in ['min', 'max', 'sum', 'mean', 'std', 'skew', 'kurtosis']:
            df[c+agg] = eval('df[col].' + agg + '(axis=1)')
            df[c+'a'+agg] = eval('df[col].abs().' + agg + '(axis=1)')
    return df

train_data = features(train_data).fillna(0)
test_data = features(test_data).fillna(0)
print(train_data.shape, test_data.shape)

(4658147, 71) (2505542, 70)


In [30]:
train_data.drop(['type', 'mulliken_charge', 'fc', 'sd', 'pso', 'dso'], axis=1, inplace=True)
test_data.drop(['type', 'mulliken_charge', 'fc', 'sd', 'pso', 'dso'], axis=1, inplace=True)

In [ ]:
col = [c for c in train.columns if c not in ['scalar_coupling_constant']]
reg = ExtraTreesRegressor(n_jobs=-1, n_estimators=10, random_state=4)
reg.fit(train[col], train['scalar_coupling_constant'])
predictions  = reg.predict(test[col])
submission = pd.read_csv('../input/sample_submission.csv')
submission['scalar_coupling_constant'] = predictions
submission.to_csv('submission.csv', index=False)